**Phases 8-9: evaluation, and optimization**

## **Phase 8: Model Evaluation**

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [27]:
import joblib

X_train_scaled = joblib.load("../data/processed/X_train_scaled.pkl")
y_train = joblib.load("../data/processed/y_train.pkl")
X_test_scaled = joblib.load("../data/processed/X_test_scaled.pkl")
y_test = joblib.load("../data/processed/y_test.pkl")

lr_model = joblib.load("../models/logistic_regression.pkl")
svm_model = joblib.load("../models/svm.pkl")
rf_model = joblib.load("../models/random_forest.pkl")
xgb_model = joblib.load("../models/xgboost.pkl")
voting_model = joblib.load("../models/voting_classifier.pkl")

In [28]:
# Make predictions
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_svm = svm_model.predict(X_test_scaled)
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_pred_rf = rf_model.predict(X_test_scaled)
y_pred_voting = voting_model.predict(X_test_scaled)

In [29]:
# Evaluation function for cleaner code
def evaluate_model(model, y_true, y_pred, model_name, X_test_data):
    """Evaluate a single model and return metrics"""
    
    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=1)
    recall = recall_score(y_true, y_pred, pos_label=1)
    f1 = f1_score(y_true, y_pred, pos_label=1)
    
    return {
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
    }

# Evaluate all models
print("\n" + "="*60)
print("📊 MODEL EVALUATION RESULTS")
print("="*60)

# Evaluate each model
lr_results = evaluate_model(lr_model, y_test, y_pred_lr, "Logistic Regression", X_test_scaled)
svm_results = evaluate_model(svm_model, y_test, y_pred_svm, "SVM", X_test_scaled)
xgb_results = evaluate_model(xgb_model, y_test, y_pred_xgb, "XGBoost", X_test_scaled)
rf_results = evaluate_model(rf_model, y_test, y_pred_rf, "Random Forest", X_test_scaled)
voting_results = evaluate_model(voting_model, y_test, y_pred_voting, "Voting Classifier", X_test_scaled)

# Display detailed results for each model
models_to_evaluate = [lr_results, svm_results, xgb_results, rf_results, voting_results]

for model_result in models_to_evaluate:
    print(f"\n{model_result['Model'].upper()}")
    print("-" * 40)
    print(f"Recall:    {model_result['Recall']:.4f}")
    # print(f"Accuracy:  {model_result['Accuracy']:.4f}")
    # print(f"Precision: {model_result['Precision']:.4f}")
    # print(f"F1 Score:  {model_result['F1 Score']:.4f}")
    # print(f"ROC AUC:   {model_result['ROC AUC']:.4f}")
    
    # Get predictions for classification report
    if model_result['Model'] == "Logistic Regression":
        y_pred_for_report = y_pred_lr
    elif model_result['Model'] == "SVM":
        y_pred_for_report = y_pred_svm
    elif model_result['Model'] == "XGBoost":
        y_pred_for_report = y_pred_xgb
    else:  # Random Forest
        y_pred_for_report = y_pred_rf
    
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred_for_report))


📊 MODEL EVALUATION RESULTS

LOGISTIC REGRESSION
----------------------------------------
Recall:    0.9286

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97        72
           1       0.97      0.93      0.95        42

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114


SVM
----------------------------------------
Recall:    0.9048

Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        72
           1       1.00      0.90      0.95        42

    accuracy                           0.96       114
   macro avg       0.97      0.95      0.96       114
weighted avg       0.97      0.96      0.96       114


XGBOOST
----------------------------------------
Recall:    0.9048

Classification Report:
              precision    recall 

## **Phase 9: Model Optimization**

In [ ]:
# Simple SVM threshold optimization
print("\n" + "="*60)
print("🎯 Voting Model THRESHOLD OPTIMIZATION - FINDING BEST RECALL")
print("="*60)

# Get SVM probabilities (using the same data format as your existing code)
voting_probs = voting_model.predict_proba(X_test_scaled)[:, 1]

# Test different thresholds and find best recall
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
best_recall = 0
best_threshold = 0.5
best_metrics = {}

print("Threshold | Recall  | Precision | F1-Score")
print("-" * 45)

for threshold in thresholds:
    # Apply threshold
    predictions = (voting_probs >= threshold).astype(int)
    pred_labels = [1 if pred == 1 else 0 for pred in predictions]
    
    # Calculate metrics
    recall = recall_score(y_test, pred_labels, pos_label=1)
    precision = precision_score(y_test, pred_labels, pos_label=1)
    f1 = f1_score(y_test, pred_labels, pos_label=1)
    
    print(f"{threshold:^9} | {recall:.4f}  | {precision:.4f}    | {f1:.4f}")
    
    # Track best recall
    if recall > best_recall:
        best_recall = recall
        best_threshold = threshold
        best_metrics = {'recall': recall, 'precision': precision, 'f1': f1}

print(f"\n🏆 BEST Voting Model THRESHOLD: {best_threshold}")
print(f"   Recall: {best_metrics['recall']:.4f}")
print(f"   Precision: {best_metrics['precision']:.4f}")
print(f"   F1-Score: {best_metrics['f1']:.4f}")

# Show high-risk customers
high_risk_count = (svm_probs >= best_threshold).sum()
print(f"\n🔴 HIGH-RISK PATIENTS (Probability >= {best_threshold}): {high_risk_count}")

print("✅ Voting Model threshold optimization completed!")
print(f"💡 Use threshold {best_threshold} for production deployment")


🎯 SVM THRESHOLD OPTIMIZATION - FINDING BEST RECALL
Threshold | Recall  | Precision | F1-Score
---------------------------------------------
   0.1    | 0.9762  | 0.8200    | 0.8913
   0.2    | 0.9762  | 0.9762    | 0.9762
   0.3    | 0.9762  | 1.0000    | 0.9880
   0.4    | 0.9762  | 1.0000    | 0.9880
   0.5    | 0.9286  | 1.0000    | 0.9630
   0.6    | 0.8810  | 1.0000    | 0.9367
   0.7    | 0.8333  | 1.0000    | 0.9091
   0.8    | 0.7857  | 1.0000    | 0.8800
   0.9    | 0.7619  | 1.0000    | 0.8649

🏆 BEST SVM THRESHOLD: 0.1
   Recall: 0.9762
   Precision: 0.8200
   F1-Score: 0.8913

🔴 HIGH-RISK CUSTOMERS (Probability >= 0.1): 50
✅ SVM threshold optimization completed!
💡 Use threshold 0.1 for production deployment


In [44]:
import json
import os
import pandas as pd

def save_performance_metrics(best_threshold, recall, precision, f1_score, algorithm="Voting Model"):
    """
    Save the best model's performance metrics to a JSON file
    
    Parameters:
    - best_threshold: Optimal threshold value
    - recall, precision, f1_score, accuracy: Core metrics
    - specificity, roc_auc: Optional additional metrics
    - algorithm: Model algorithm name
    """
    
    # Create metrics dictionary
    metrics = {
        "algorithm": algorithm,
        "best_threshold": best_threshold,
        "core_metrics": {
            "recall": round(recall, 4),
            "precision": round(precision, 4),
            "f1_score": round(f1_score, 4)
        },
        "training_info": {
            "date": pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
            "dataset_size": len(X_train) + len(X_test)
        }
    }
    
    # Create models directory if it doesn't exist
    os.makedirs("models", exist_ok=True)
    
    # Save to JSON file
    metrics_path = "models/performance_metrics.json"
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=2)
    
    print(f"✅ Performance metrics saved to: {metrics_path}")
    return metrics_path

# Use this after you find your best threshold in your notebook:
# Example usage:
# save_performance_metrics(
#     best_threshold=0.1,
#     recall=0.9762,
#     precision=0.8200,
#     f1_score=0.8913,
#     accuracy=0.8947,
#     specificity=0.8235,
#     roc_auc=0.9234,
#     algorithm="SVM"
# )
save_performance_metrics(
    best_threshold=best_threshold,
    recall=best_metrics['recall'],
    precision=best_metrics['precision'],
    f1_score=best_metrics['f1'],
    algorithm="Voting Model"
)

NameError: name 'X_train' is not defined

## Saving the Best Model

In [ ]:
# Save the optimized SVM model with best threshold
print("\n" + "="*60)
print("💾 SAVING OPTIMIZED VOTING MODEL")
print("="*60)

# Create models directory if it doesn't exist
models_path = "../models"
os.makedirs(models_path, exist_ok=True)

# Save the Voting model
best_model_path = f"{models_path}/best_model.pkl"
joblib.dump(voting_model, best_model_path)
print(f"✅ Best model saved to: {best_model_path}")

# Save the best threshold value
threshold_path = f"{models_path}/best_threshold.txt"
with open(threshold_path, 'w') as f:
    f.write(str(best_threshold))
print(f"✅ Best threshold saved to: {threshold_path}")

# Save the scaler for preprocessing new data
scaler = StandardScaler()
scaler_path = f"{models_path}/scaler.pkl"
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to: {scaler_path}")

print(f"\n🎯 Model deployment ready!")
print(f"   - Use threshold: {best_threshold}")
print(f"   - Model file: {best_model_path}")
print(f"   - For new predictions: load model and apply threshold {best_threshold}")


💾 SAVING OPTIMIZED VOTING MODEL
✅ Best model saved to: ../models/best_model.pkl
✅ Best threshold saved to: ../models/best_threshold.txt
✅ Scaler saved to: ../models/scaler.pkl

🎯 Model deployment ready!
   - Use threshold: 0.1
   - Model file: ../models/best_model.pkl
   - For new predictions: load model and apply threshold 0.1
